In [1]:
# %load_ext autorelaod
# %autoreload 2

import sys
sys.path.append('./../RWKV-v4neo/src')
import torch 
from torch.utils import data
from enwik8_dataset import EnWik8
import json
import argparse
import os
from torch import nn

/system/apps/userenv/beck/rwkv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_json(path):
    with open(path, 'r') as f:
        return json.load(f)
def set_environ(env_vars: dict):
    for k, v in env_vars.items():
        os.environ[k] = v

In [3]:
data_path = '/system/user/beck/pwbeck/projects/rwkv/RWKV-LM/data/enwik8'
args = load_json('./debug_args.json')
args = argparse.Namespace(**args)
env_vars = load_json('./debug_environ.json')
set_environ(env_vars)

In [4]:
batch_size = 12

In [5]:
dataset = EnWik8(datafile=data_path)
dataloader = data.DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True, drop_last=True)
args.vocab_size = dataset.vocab_size

Data has 99621832 tokens, 6064 vocab size.


In [6]:
batch = next(iter(dataloader))
x, y = batch
x.shape, y.shape

(torch.Size([12, 512]), torch.Size([12, 512]))

### RWKV-v4neo model

In [7]:
from model import RWKV, Block, RWKV_ChannelMix, RWKV_TimeMix
model = RWKV(args)

RWKV_MY_TESTING 


Using /system/user/beck/.cache/torch_extensions/py310_cu117 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /system/user/beck/.cache/torch_extensions/py310_cu117/wkv_512/build.ninja...
Building extension module wkv_512...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module wkv_512...


In [8]:
output = model(x)

In [9]:
output.shape

torch.Size([12, 512, 6064])

### Reimplementation of RWKV-v4neo model

In [7]:
from rwkv_model import RWKV, RWKVConfig

Using /system/user/beck/.cache/torch_extensions/py310_cu117 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /system/user/beck/.cache/torch_extensions/py310_cu117/wkv/build.ninja...
Building extension module wkv...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module wkv...


In [8]:
config = RWKVConfig(embedding_dim=512,
                    attention_dim=512,
                    ffn_dim=2048,
                    num_blocks=6,
                    vocab_size=dataset.vocab_size, #6064
                    context_len=512)


In [9]:
mymodel = RWKV(config).to(device='cuda')


In [10]:
x = x.to(device='cuda')
out = mymodel(x)

In [11]:
out.shape

torch.Size([12, 512, 6064])